In [ ]:
import numpy as np
import scipy.io as sio
import cv2
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report

# 📌 Load Hyperspectral Image Data
def load_hsi_data(file_path, label_path):
    if file_path.endswith(".mat"):
        data = sio.loadmat(file_path)['data']
        labels = sio.loadmat(label_path)['labels']
    elif file_path.endswith(".npy"):
        data = np.load(file_path)
        labels = np.load(label_path)
    else:
        raise ValueError("Unsupported file format")

    return data, labels

# 📌 Preprocess the Data
def preprocess_hsi(data, labels):

    h, w, c = data.shape
    data = data.reshape(-1, c)
    labels = labels.flatten()

    mask = labels > 0
    data = data[mask]
    labels = labels[mask]
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test

# 📌 Train SVM Model
def train_svm(X_train, y_train):

    svm = SVC(kernel='rbf', C=1.0, gamma='scale')
    svm.fit(X_train, y_train)
    return svm

# 📌 Evaluate Model
def evaluate_svm(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"✅ SVM Accuracy: {acc * 100:.2f}%")
    print("📌 Classification Report:\n", report)

# 📌 Run the SVM Pipeline
if __name__ == "__main__":
    file_path = "hsi_data.mat"
    label_path = "hsi_labels.mat"

    print("Loading Data...")
    data, labels = load_hsi_data(file_path, label_path)

    print("Preprocessing Data...")
    X_train, X_test, y_train, y_test = preprocess_hsi(data, labels)

    print("Training SVM Model...")
    svm_model = train_svm(X_train, y_train)

    print("Evaluating Model...")
    evaluate_svm(svm_model, X_test, y_test)
